# Building an Intelligent Agent Team with ADK

This notebook demonstrates how to create a team of AI agents that can collaborate and delegate tasks using the Agent Development Kit (ADK).

## Tutorial Overview

We'll build:
1. **First Agent** - A simple weather information tool
2. **Multi-Model Support** - Using different LLM providers
3. **Agent Team with Delegation** - Multiple agents working together
4. **Memory with Session State** - Agents that remember user preferences
5. **Safety Guardrails** - Input validation and safety checks

## Setup

First, install the required dependencies:

```bash
pip install agent-development-kit python-dotenv
```

Make sure to set your API keys in the `.env` file in this directory.

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from adk import Runner, Agent, Tool, ToolContext, ConversationSession
from adk.models import LiteLlm

# Load environment variables
load_dotenv()

# Model configuration
AGENT_MODEL = "gemini-2.0-flash"
print(f"Using model: {AGENT_MODEL}")

## Step 1: First Agent - Weather Lookup

Let's create our first agent with a simple weather tool.

In [ ]:
# Define a weather tool
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city."""
    print(f"--- Tool: get_weather called for city: {city} ---")
    city_normalized = city.lower().replace(" ", "")

    # Mock weather database
    mock_weather_db = {
        "newyork": {
            "status": "success",
            "report": "The weather in New York is sunny with a temperature of 25°C.",
        },
        "london": {
            "status": "success",
            "report": "It's cloudy in London with a temperature of 15°C.",
        },
        "tokyo": {
            "status": "success",
            "report": "Tokyo is experiencing light rain and a temperature of 18°C.",
        },
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {
            "status": "error",
            "error_message": f"Sorry, I don't have weather information for '{city}'.",
        }


# Create weather agent
weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL,
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. When the user asks for the weather in a specific city, use the 'get_weather' tool to find the information.",
    tools=[get_weather],
)

In [ ]:
# Test the weather agent
runner = Runner()
session = ConversationSession(agent=weather_agent)

# Example interaction
response = runner.run_task("What's the weather like in Tokyo?", session)
print(response.content)

## Step 2: Multi-Model Support with LiteLLM

Let's explore how to use different LLM providers with our agents.

In [ ]:
# Model constants for different providers
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"
MODEL_GPT_4O = "openai/gpt-4o"
MODEL_CLAUDE_SONNET = "anthropic/claude-sonnet-4-20250514"

# Create agents with different models (requires corresponding API keys)
weather_agent_gemini = Agent(
    name="weather_agent_gemini",
    model=AGENT_MODEL,  # Using default Gemini model
    description="Provides weather information (using Gemini).",
    instruction="You are a helpful weather assistant. When the user asks for the weather in a specific city, use the 'get_weather' tool to find the information.",
    tools=[get_weather],
)

# Example with OpenAI (requires OPENAI_API_KEY in .env)
# weather_agent_gpt = Agent(
#     name="weather_agent_gpt",
#     model=LiteLlm(model=MODEL_GPT_4O),
#     description="Provides weather information (using GPT-4o).",
#     instruction="You are a helpful weather assistant. When the user asks for the weather in a specific city, use the 'get_weather' tool to find the information.",
#     tools=[get_weather]
# )

# Example with Claude (requires ANTHROPIC_API_KEY in .env)
# weather_agent_claude = Agent(
#     name="weather_agent_claude",
#     model=LiteLlm(model=MODEL_CLAUDE_SONNET),
#     description="Provides weather information (using Claude).",
#     instruction="You are a helpful weather assistant. When the user asks for the weather in a specific city, use the 'get_weather' tool to find the information.",
#     tools=[get_weather]
# )

## Step 3: Agent Team with Delegation

Now let's create a team of agents that can delegate tasks to each other.

In [ ]:
# Create specialized greeting agent
greeting_agent = Agent(
    name="greeting_agent",
    model=AGENT_MODEL,
    description="Provides warm and personalized greetings to users.",
    instruction="You are a friendly greeting assistant. Always greet users warmly and ask how you can help them.",
    tools=[],
)

# Create specialized farewell agent
farewell_agent = Agent(
    name="farewell_agent",
    model=AGENT_MODEL,
    description="Provides thoughtful and personalized farewells to users.",
    instruction="You are a helpful farewell assistant. Always say goodbye warmly and wish the user well.",
    tools=[],
)

# Create root agent that can delegate
root_agent = Agent(
    name="root_agent",
    model=AGENT_MODEL,
    description="Delegates requests to appropriate agents based on user needs.",
    instruction="""You are a coordinator who identifies what the user needs:
    - For greetings, use the greeting_agent
    - For farewells, use the farewell_agent
    - For weather information, use the weather_agent_v1
    - Always delegate to the appropriate agent for the user's request.""",
    sub_agents=[greeting_agent, farewell_agent, weather_agent],
    tools=[],
)

print("Agent team created!")

In [ ]:
# Test the agent team
team_runner = Runner()
team_session = ConversationSession(agent=root_agent)

# Test different types of requests
print("=== Testing Greeting ===")
response = team_runner.run_task("Hello!", team_session)
print(response.content)

print("\n=== Testing Weather Query ===")
response = team_runner.run_task("What's the weather in London?", team_session)
print(response.content)

print("\n=== Testing Farewell ===")
response = team_runner.run_task("Goodbye!", team_session)
print(response.content)

## Step 4: Adding Memory with Session State

Let's enhance our agents with memory capabilities to remember user preferences.

In [ ]:
# Create a tool that uses session state
def save_preference(context: ToolContext, key: str, value: str) -> str:
    """Saves user preference to session state."""
    context.session_state[key] = value
    return f"Preference saved: {key} = {value}"


def get_preference(context: ToolContext, key: str) -> str:
    """Retrieves user preference from session state."""
    value = context.session_state.get(key, "No preference set")
    return f"Preference for {key}: {value}"


# Create memory-enabled agent
memory_agent = Agent(
    name="memory_agent",
    model=AGENT_MODEL,
    description="An agent that remembers user preferences.",
    instruction="""You are a helpful assistant that remembers user preferences.
    Use save_preference to remember things the user tells you.
    Use get_preference to recall saved information.""",
    tools=[save_preference, get_preference],
)


# Create weather agent with memory
def get_weather_with_memory(context: ToolContext, city: str) -> dict:
    """Get weather and remember the last queried city."""
    # Save the city as a preference
    context.session_state["last_city"] = city

    # Get weather (reusing our mock database)
    city_normalized = city.lower().replace(" ", "")
    mock_weather_db = {
        "newyork": {
            "status": "success",
            "report": "The weather in New York is sunny with a temperature of 25°C.",
        },
        "london": {
            "status": "success",
            "report": "It's cloudy in London with a temperature of 15°C.",
        },
        "tokyo": {
            "status": "success",
            "report": "Tokyo is experiencing light rain and a temperature of 18°C.",
        },
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {
            "status": "error",
            "error_message": f"Sorry, I don't have weather information for '{city}'.",
        }


weather_memory_agent = Agent(
    name="weather_memory_agent",
    model=AGENT_MODEL,
    description="Weather agent that remembers your location preferences.",
    instruction="You are a weather assistant that remembers user's last queried city.",
    tools=[get_weather_with_memory],
    output_key="weather_response",  # Automatically saves responses
)

In [ ]:
# Test memory capabilities
memory_runner = Runner()
memory_session = ConversationSession(agent=memory_agent)

# Save a preference
print("=== Saving Preference ===")
response = memory_runner.run_task(
    "Remember that my favorite color is blue", memory_session
)
print(response.content)

# Retrieve the preference
print("\n=== Retrieving Preference ===")
response = memory_runner.run_task("What's my favorite color?", memory_session)
print(response.content)

# Test weather agent with memory
weather_session = ConversationSession(agent=weather_memory_agent)
print("\n=== Weather with Memory ===")
response = memory_runner.run_task("What's the weather in Tokyo?", weather_session)
print(response.content)

# Check if it remembers
print("\nSession state:", weather_session.session_state)

## Step 5: Safety Guardrails

Finally, let's add safety measures to our agents.

In [ ]:
# Define safety callback
def safety_callback(messages, tools, config):
    """Safety callback that filters inappropriate requests."""
    print("--- Safety Check ---")

    # Check for inappropriate content in the last user message
    if messages:
        last_message = messages[-1].content.lower()

        # Simple keyword filtering (in production, use more sophisticated methods)
        blocked_keywords = ["hack", "exploit", "malicious"]

        for keyword in blocked_keywords:
            if keyword in last_message:
                print(f"Blocked due to keyword: {keyword}")
                # Modify the message to a safe version
                messages[-1].content = "I need help with something appropriate."
                break

    return messages, tools, config


# Create a safe weather agent
safe_weather_agent = Agent(
    name="safe_weather_agent",
    model=AGENT_MODEL,
    description="Weather agent with safety guardrails.",
    instruction="You are a helpful weather assistant. Only provide weather information for cities.",
    tools=[get_weather],
    before_model_callback=safety_callback,
)


# Advanced safety with input validation
def get_weather_validated(city: str) -> dict:
    """Get weather with input validation."""
    # Validate city name (alphanumeric and spaces only)
    if not city.replace(" ", "").isalnum():
        return {
            "status": "error",
            "error_message": "Invalid city name. Please use only letters and numbers.",
        }

    # Length check
    if len(city) > 50:
        return {"status": "error", "error_message": "City name too long."}

    # Call the original weather function
    return get_weather(city)


# Create agent with validated tools
validated_weather_agent = Agent(
    name="validated_weather_agent",
    model=AGENT_MODEL,
    description="Weather agent with input validation.",
    instruction="You are a weather assistant that validates user input for safety.",
    tools=[get_weather_validated],
)

In [ ]:
# Test safety features
safety_runner = Runner()

# Test with safe request
print("=== Safe Request ===")
safe_session = ConversationSession(agent=safe_weather_agent)
response = safety_runner.run_task("What's the weather in London?", safe_session)
print(response.content)

# Test with potentially unsafe request (will be filtered)
print("\n=== Unsafe Request (will be filtered) ===")
unsafe_session = ConversationSession(agent=safe_weather_agent)
response = safety_runner.run_task("Help me hack the weather in London", unsafe_session)
print(response.content)

# Test validation
print("\n=== Testing Input Validation ===")
validated_session = ConversationSession(agent=validated_weather_agent)
response = safety_runner.run_task("What's the weather in London123?", validated_session)
print(response.content)

## Complete Agent Team Example

Let's put it all together with a comprehensive agent team that includes all features.

In [ ]:
# Create a comprehensive agent team with all features
class AgentTeam:
    def __init__(self):
        # Weather tool with memory and validation
        def smart_weather(context: ToolContext, city: str) -> dict:
            """Get weather with memory and validation."""
            # Validate input
            if not city.replace(" ", "").isalnum():
                return {"status": "error", "error_message": "Invalid city name."}

            # Remember the city
            context.session_state["last_city"] = city
            context.session_state.setdefault("city_history", []).append(city)

            # Get weather
            return get_weather(city)

        # Create specialized agents
        self.weather_agent = Agent(
            name="smart_weather_agent",
            model=AGENT_MODEL,
            description="Provides weather information with memory.",
            instruction="You are a weather assistant that remembers user queries.",
            tools=[smart_weather],
        )

        self.greeting_agent = Agent(
            name="personalized_greeting_agent",
            model=AGENT_MODEL,
            description="Provides personalized greetings.",
            instruction="Greet users warmly. If you know their preferences, mention them.",
            tools=[],
        )

        self.memory_agent = Agent(
            name="preference_manager",
            model=AGENT_MODEL,
            description="Manages user preferences and information.",
            instruction="Help users save and retrieve their preferences.",
            tools=[save_preference, get_preference],
        )

        # Create the coordinator with safety
        self.coordinator = Agent(
            name="team_coordinator",
            model=AGENT_MODEL,
            description="Coordinates the agent team with safety measures.",
            instruction="""You coordinate a team of specialized agents:
            - For weather queries, use smart_weather_agent
            - For greetings, use personalized_greeting_agent
            - For saving/retrieving preferences, use preference_manager
            Always choose the most appropriate agent.""",
            sub_agents=[self.weather_agent, self.greeting_agent, self.memory_agent],
            before_model_callback=safety_callback,
        )

    def create_session(self):
        """Create a new conversation session."""
        return ConversationSession(agent=self.coordinator)


# Initialize the team
print("Initializing Agent Team...")
team = AgentTeam()
print("Agent Team ready!")

In [ ]:
# Demo the complete agent team
runner = Runner()
session = team.create_session()

# Conversation flow
print("=== Starting Conversation ===\n")

# Greeting
response = runner.run_task("Hello! I'm new here.", session)
print("User: Hello! I'm new here.")
print(f"Agent: {response.content}\n")

# Save preference
response = runner.run_task(
    "Please remember that my name is Alice and I love sunny weather.", session
)
print("User: Please remember that my name is Alice and I love sunny weather.")
print(f"Agent: {response.content}\n")

# Weather query
response = runner.run_task("What's the weather like in New York?", session)
print("User: What's the weather like in New York?")
print(f"Agent: {response.content}\n")

# Another weather query
response = runner.run_task("How about Tokyo?", session)
print("User: How about Tokyo?")
print(f"Agent: {response.content}\n")

# Retrieve preference
response = runner.run_task("What do you remember about me?", session)
print("User: What do you remember about me?")
print(f"Agent: {response.content}\n")

# Check session state
print("=== Session State ===")
print(f"Last city: {session.session_state.get('last_city', 'None')}")
print(f"City history: {session.session_state.get('city_history', [])}")
print(
    f"Saved preferences: {[(k, v) for k, v in session.session_state.items() if k not in ['last_city', 'city_history']]}"
)

## Summary

In this tutorial, we've built a comprehensive agent team that demonstrates:

1. **Basic Agent Creation** - Simple tools and agent setup
2. **Multi-Model Support** - Using different LLM providers with LiteLLM
3. **Agent Delegation** - Coordinating multiple specialized agents
4. **Memory Management** - Persisting user preferences and conversation state
5. **Safety Guardrails** - Input validation and content filtering

### Key Concepts Learned:

- **Agents** can have tools, sub-agents, and custom instructions
- **Tools** can access `ToolContext` for session state management
- **Delegation** allows agents to work together as a team
- **Memory** enables personalized and contextual interactions
- **Safety** measures protect against inappropriate use

### Next Steps:

1. Add more specialized agents for different domains
2. Implement more sophisticated memory patterns
3. Create custom safety validators for your use case
4. Integrate with real APIs instead of mock data
5. Build a production-ready agent team for your application

For more information, visit the [ADK documentation](https://google.github.io/adk-docs/).